# Sampling

In [1]:
import os
import json
import random
import shutil
import tqdm

from PIL import Image


In [2]:
label_kr = {
    'c_1' :0, #'종이',
    'c_2_01': 1, #'종이팩',
    'c_2_02': 2, #'종이컵',
    'c_3': 3, #'캔류',
    'c_4_01_02': 4, #'재사용 유리(소주병 + 맥주병)',
    'c_4_02_01_02': 5, #'갈색 유리',
    'c_4_02_02_02': 6, #'녹색 유리',
    'c_4_02_03_02': 7, #'백색 유리',
    'c_4_03': 8, #'기타 유리',
    'c_5_02': 9, #'페트',
    'c_6': 10, #'플라스틱',
    'c_7': 11, #'비닐',
    ## 이 위에는 재활용
    ## 밑에는 일반쓰레기
    'c_1_01': 12, #'종이 + 이물질',
    'c_2_02_01': 13, #'종이컵 + 이물질',
    'c_3_01': 14, #'캔 + 이물질',
    'c_4_03_01': 15, #'기타 유리 + 이물질',
    'c_5_01_01': 16, #'페트 + 이물질 + 다중포장재',
    'c_5_02_01' : 17, #'페트+이물질',
    'c_6_01': 18, #'플라스틱 + 이물질',
    'c_7_01': 19, #'비닐 + 이물질',

    # 다중포장재
    'c_4_01_01': 20, #'재사용 유리(소주병+맥주병) + 다중 포장재',
    'c_4_02_01_01': 21, #'갈색 유리 + 다중포장재',
    'c_4_02_02_01': 22, #'녹색 유리 + 다중포장재',
    'c_4_02_03_01': 23, #'백색 유리 + 다중포장재',
    'c_5_01': 24, #'페트 + 다중포장재',
    'c_8_01': 25, #'흰색 스티로폼',
    'c_8_02': 26, #'컬러 스티로폼',
    'c_8_01_01': 27, #'스티로폼 + 이물질',
    'c_9': 28, #'건전지'
}

# 무결성 검사

In [23]:
images_path = './mydata/images_folder'
json_path = './mydata/jsons_folder'

images_list = os.listdir(images_path)
json_list = os.listdir(json_path)

print('이미지 형식 jpg로 통일')
for file_name in tqdm.tqdm(images_list):
    if file_name.endswith('.JPG'):
        new_file_name = file_name.replace('.JPG', '.jpg')
        old_file_path = os.path.join(images_path, file_name)
        new_file_path = os.path.join(images_path, new_file_name)
        os.rename(old_file_path, new_file_path)

for file_list in [images_list,json_list]:
    for idx , file_name in tqdm.tqdm(enumerate(file_list)):
        file_list[idx] = os.path.splitext(file_name)[0]

cnt = 0
for file_name in images_list:
    if file_name not in json_list: 
        os.remove(os.path.join(images_path,file_name+'.jpg'))
        cnt += 1
print(f'삭제된 이미지 파일 개수 : {cnt}')

cnt = 0
for file_name in json_list:
    if file_name not in images_list: 
        os.remove(os.path.join(json_path,file_name+'.json'))
        cnt += 1
print(f'삭제된 json 파일 개수 : {cnt}')

images_list = os.listdir(images_path)
json_list = os.listdir(json_path)
for file_list in [images_list,json_list]:
    for idx , file_name in tqdm.tqdm(enumerate(file_list)):
        file_list[idx] = os.path.splitext(file_name)[0]
print(f'최종 이미지 개수 : {len(images_list)} / json 개수 : {len(json_list)}')

inter_list = set(images_list+json_list)
print(f'공통된 파일 개수 : {len(inter_list)}')

이미지 형식 jpg로 통일


100%|██████████| 151440/151440 [00:00<00:00, 4207194.51it/s]
151440it [00:00, 912111.60it/s]
151440it [00:00, 901262.39it/s]


삭제된 이미지 파일 개수 : 0
삭제된 json 파일 개수 : 0


151440it [00:00, 912162.68it/s]
151440it [00:00, 909455.28it/s]

최종 이미지 개수 : 151440 / json 개수 : 151440
공통된 파일 개수 : 151440


# Uniform 분포

In [10]:
# 폴더 명 확인인

max_val = 1230
min_val = 1200
AB_path = './mydata/jsons_folder' # json파일만 들어있음
to_path = './mydata/uniform_data/train/labels'
seed = 1111

special = [4 , 6 , 19, 22,27]

json_list=[]
AB_label_count=[0 for _ in range(29)]
break_x = False

AB_label_list = os.listdir(AB_path)
random.Random(seed).shuffle(AB_label_list) # random shuffling
for file_name in tqdm.tqdm(AB_label_list):
    temp_list = [0 for _ in range(29)]
    with open(os.path.join(AB_path,file_name), 'r',encoding="UTF-8") as file:
        data = json.load(file)
    for dictionary in data["objects"]:
        temp_list[label_kr[dictionary['class_name']]] += 1
    if sum(temp_list[idx] for idx in special) and all((AB_label_count[idx] + temp_list[idx] < min_val) for idx in range(29)):
        for i in range(29):
            AB_label_count[i] += temp_list[i]
        json_list.append(file_name)
        shutil.move(os.path.join(AB_path,file_name) , os.path.join(to_path,file_name))
    
print(AB_label_count)

for patience in range(3):
    AB_label_list = os.listdir(AB_path)
    random.Random(seed).shuffle(AB_label_list) # random shuffling

    for file_name in tqdm.tqdm(AB_label_list):
        if all(AB_label_count[i]<min_val for i in range(29)):
            with open(os.path.join(AB_path,file_name), 'r',encoding="UTF-8") as file:
                data = json.load(file)
            for dictionary in data["objects"]:
                AB_label_count[label_kr[dictionary['class_name']]] += 1
            json_list.append(file_name)
            shutil.move(os.path.join(AB_path,file_name) , os.path.join(to_path,file_name))
            pass

        elif any(AB_label_count[i]<min_val for i in range(29)):
            need_list = [1 if x < min_val else 0 for x in AB_label_count]
            temp_list = [0 for _ in range(29)]
            with open(os.path.join(AB_path,file_name), 'r',encoding="UTF-8") as file:
                data = json.load(file)
            for dictionary in data["objects"]:
                temp_list[label_kr[dictionary['class_name']]] += 1

            go_x = True
            cnt = 0
            for i in range(29):
                if need_list[i]:
                    if temp_list[i] > 0:
                        cnt += 1
                else:
                    if temp_list[i] > 2*patience or (AB_label_count[i] > max_val + 2*(patience**2)) :
                        go_x = False
                        break

                        
            if go_x and cnt:
                for i in range(29):
                    AB_label_count[i] +=  temp_list[i]
                json_list.append(file_name)
                shutil.move(os.path.join(AB_path,file_name) , os.path.join(to_path,file_name))
            
        if all(AB_label_count[i]>=min_val for i in range(29)):
            break_x = True
            break
     
    if break_x:
        break     
    print(AB_label_count) # samplinge된 클래스별 개수
print(AB_label_count) 
print(len(json_list)) # sampling된 이미지 개수
moved_lsit = os.listdir(to_path)
print(len(moved_lsit))

100%|██████████| 144913/144913 [00:36<00:00, 3965.02it/s]


[567, 56, 87, 477, 1171, 3, 1199, 14, 21, 199, 286, 728, 70, 87, 216, 50, 167, 146, 327, 1199, 176, 159, 1199, 141, 261, 101, 0, 1199, 65]


100%|██████████| 139136/139136 [00:47<00:00, 2942.42it/s]


[1200, 1200, 1200, 1200, 1171, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200]


100%|██████████| 118986/118986 [00:26<00:00, 4539.30it/s]


[1200, 1200, 1200, 1201, 1182, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1205, 1200, 1200, 1201, 1201, 1200, 1200, 1200, 1205, 1200]


100%|██████████| 118975/118975 [00:26<00:00, 4497.56it/s]

[1200, 1200, 1200, 1201, 1182, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1205, 1200, 1200, 1201, 1201, 1200, 1200, 1200, 1205, 1200]
[1200, 1200, 1200, 1201, 1182, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1205, 1200, 1200, 1201, 1201, 1200, 1200, 1200, 1205, 1200]
25938
25938


In [12]:
# 위에 코드 재실행할 때 사용
to_path = './mydata/uniform_data/val/labels'
json_list = os.listdir(to_path)
cnt = 0
for file_name in json_list:
    shutil.copy(os.path.join(to_path,file_name) , os.path.join(AB_path,file_name))
    #shutil.move(os.path.join(to_path,file_name) , os.path.join(AB_path,file_name))
    cnt += 1
print(cnt)

6527


In [ ]:
# Uniform 분포/ c폴더 4번 인덱스는 최대 값이 1182임
[1200, 1200, 1200, 1200, 1182, 1200, 1200, 1200, 1200, 1200, 1200, 1197, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1204, 1200, 1200, 1201, 1201, 1200, 1200, 1200, 1205, 1200]
25937

[300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300]
6527

In [ ]:
# Natural 분포/ c폴더 합하면 180,794개
[5125, 830, 894, 3604, 249, 531, 268, 543, 330, 977, 4349, 6806, 602, 675, 1136, 1086, 1130, 779, 3490, 375, 1167, 804, 334, 823, 2374, 642, 966, 264, 1078]
25000개

[1243, 180, 182, 810, 66, 110, 80, 114, 73, 245, 1014, 1679, 170, 160, 259, 261, 253, 173, 819, 100, 288, 199, 80, 183, 622, 144, 265, 73, 253]
6000개

# 자연 분포

In [13]:
from_path = './mydata/jsons_folder'
json_list = os.listdir(from_path)
print(f'총 데이터 : {len(json_list)}개')

random.seed(1234)  # 랜덤 시드 설정
train_list = random.sample(json_list, 25000)
to_path = './mydata/natural_data/train/labels'
cnt = 0
for file_name in train_list:
    shutil.move(os.path.join(from_path,file_name) , os.path.join(to_path,file_name))
    cnt += 1
print(f'train data : {cnt}개')


json_list = os.listdir(from_path)

random.seed(1234)  # 랜덤 시드 설정
train_list = random.sample(json_list, 6000)
to_path = './mydata/natural_data/val/labels'
cnt = 0
for file_name in train_list:
    shutil.move(os.path.join(from_path,file_name) , os.path.join(to_path,file_name))
    cnt += 1
print(f'valid data : {cnt}개')

총 데이터 : 151440개
train data : 25000개
valid data : 6000개


# 이미지 Resize

In [16]:
# 필요없는 이미지 제거
label_path_list = ['./mydata/uniform_data/train/labels',
              './mydata/uniform_data/val/labels',
              './mydata/natural_data/train/labels',
              './mydata/natural_data/val/labels']

from_image_path = './mydata/images_folder'

label_list = []
for label_path in label_path_list:
    t_list = os.listdir(label_path)
    label_list += t_list
print(f'json 데이터 개수 : {len(label_list)}')                  # 62687개가 되야함
print(f'중복 없는 데이터 개수 : {len(set(label_list))}')

for i,label_name in enumerate(label_list):
    label_list[i] = os.path.splitext(label_name)[0]

image_list = os.listdir(from_image_path)
for image_name in tqdm.tqdm(image_list):
    if os.path.splitext(image_name)[0] in label_list:
        pass
    else:
        os.remove(os.path.join(from_image_path,image_name))
print('필요없는 이미지 제거 완료')
print()

# 모든 json파일과 이미지 데이터가 있는지 확인
cnt = 0
image_list = os.listdir(from_image_path)

no_image_label_list = []
image_list = os.listdir(from_image_path)
for label_name in tqdm.tqdm(set(label_list)):
    if label_name + '.jpg' in image_list:
        cnt += 1
    else:
        no_image_label_list.append(label_name)
print(f'json 데이터 총 개수 : {len(set(label_list))}')
print(f'json 데이터와 매칭되는 이미지 개수 : {cnt}')



json 데이터 개수 : 63465
중복 없는 데이터 개수 : 56852


100%|██████████| 151440/151440 [04:45<00:00, 530.28it/s] 


필요없는 이미지 제거 완료



100%|██████████| 56852/56852 [00:15<00:00, 3667.16it/s]

json 데이터 총 개수 : 56852
json 데이터와 매칭되는 이미지 개수 : 56852


In [17]:
for v in sorted(no_image_label_list):
    print(v)

In [10]:
from_image_path = './mydata/images_folder'
to_image_path = './mydata/natural_data/val/images'
label_path = './mydata/natural_data/val/labels'
size = (640,640)

json_list = os.listdir(label_path)
json_list = [file for file in json_list if file.endswith('.json')]
for file_name in tqdm.tqdm(json_list):

    try:
        image = Image.open(os.path.join(from_image_path,os.path.splitext(file_name)[0]+'.jpg'))
        h , w = image.size  # 세로,가로
        resized_image = image.resize(size)
        resized_image.save(os.path.join(to_image_path,os.path.splitext(file_name)[0]+'.jpg'))

        with open(os.path.join(label_path,file_name), 'r',encoding="UTF-8") as file:
            data = json.load(file)
        extracted_objects = []      
        for dictionary in data["objects"]:                 
            class_name = label_kr[dictionary["class_name"]]
            coord_x = dictionary["annotation"]["coord"]["x"]
            coord_y = dictionary["annotation"]["coord"]["y"]
            coord_width = dictionary["annotation"]["coord"]["width"]
            coord_height = dictionary["annotation"]["coord"]["height"]
            extracted_objects.append((class_name, (coord_x+coord_width/2)/w,
                                    (coord_y+coord_height/2)/h,coord_width/w,coord_height/h))
                            
            # 추출한 값을 텍스트 파일로 저장
            text_file_name = os.path.splitext(file_name)[0] + '.txt'
            text_file_path = os.path.join(label_path, text_file_name)
            with open(text_file_path, 'w') as file:
                for obj in extracted_objects:
                    file.write(' '.join(map(str, obj)) + '\n')
        # json파일 제거
        os.remove(os.path.join(label_path,file_name))
    except:
        print(file_name)


100%|██████████| 6000/6000 [14:35<00:00,  6.85it/s]


In [18]:
distribution = ['uniform_data','natural_data']
tr_vl = ['train','val']

for dis in distribution:
    for t_v in tr_vl:
        from_path = os.path.join('./mydata',dis,t_v)
        image_path = os.path.join(from_path,'images')
        label_path = os.path.join(from_path,'labels')

        image_list = os.listdir(image_path)
        label_list = os.listdir(label_path)

        json_list = [file for file in label_list if file.endswith('.json')]
        if json_list:
            print(f'json파일이 존재합니다. : {len(json_list)}')

        label_count=[0 for _ in range(29)]

        for file_name in label_list:
            with open(os.path.join(label_path,file_name), 'r',encoding="UTF-8") as file:
                for line in file:
                    try:
                        label_count[int(line.split()[0])] += 1
                    except:
                        print(file_name)
                        break
                    
        image_list = [os.path.splitext(v)[0] for v in image_list]
        label_list = [os.path.splitext(v)[0] for v in label_list]
        image_label_list = set(image_list+label_list)
    
        print(f'{dis} 분포 - {t_v} : 이미지 개수 : {len(image_list)} / txt 개수 : {len(label_list)} / 이미지,txt쌍 개수 : {len(image_label_list)}')
        print(label_count)
        print()


uniform_data 분포 - train : 이미지 개수 : 25935 / txt 개수 : 25935 / 이미지,txt쌍 개수 : 25935
[1200, 1200, 1200, 1200, 1182, 1200, 1200, 1200, 1200, 1200, 1200, 1197, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1204, 1199, 1200, 1201, 1201, 1200, 1200, 1200, 1204, 1200]

uniform_data 분포 - val : 이미지 개수 : 6527 / txt 개수 : 6527 / 이미지,txt쌍 개수 : 6527
[300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 299, 300, 300, 300, 300, 300]

natural_data 분포 - train : 이미지 개수 : 25000 / txt 개수 : 25000 / 이미지,txt쌍 개수 : 25000
[5124, 830, 894, 3604, 249, 531, 268, 543, 330, 977, 4350, 6807, 602, 675, 1136, 1086, 1130, 779, 3490, 375, 1167, 805, 334, 824, 2374, 642, 966, 264, 1078]

natural_data 분포 - val : 이미지 개수 : 6000 / txt 개수 : 6000 / 이미지,txt쌍 개수 : 6000
[1243, 180, 182, 810, 66, 110, 80, 114, 73, 245, 1014, 1679, 170, 160, 259, 261, 253, 173, 819, 100, 288, 199, 80, 183, 622, 144, 265, 73, 253]

